In [10]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 3.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 3.0 MB/s eta 0:00:0000:0100:02


In [83]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from copy import deepcopy
from datetime import date, timedelta
from calendar import monthrange
import numpy as np
from pyswarm import pso

In [67]:
data = pd.read_csv( 'hackology_2024/axe_transactions_with_elasticity_corrected.csv' )

In [68]:
data

,transaction_datetime,customer_id,location,product_name,category,size_cm,weight_kg,quantity,unit_price,discount,total_amount,payment_method,day_of_year,cost_price,profit,transaction_month,transaction_year
0,2018-01-01 13:46:00,80,3,Small Axe,Hand Tools,35,1.5,15,30.00,44,252,Credit Card,1,12,72.0,1,2018
1,2018-01-01 09:53:00,91,12,Battle Axe,Combat Tools,95,10.0,15,120.00,44,1008,Credit Card,1,48,288.0,1,2018
2,2018-01-01 15:20:00,91,10,Medium Axe,Hand Tools,55,3.0,13,50.00,44,364,Credit Card,1,20,104.0,1,2018
3,2018-01-01 09:58:00,23,7,Small Axe,Hand Tools,35,1.5,11,30.00,44,185,Cash,1,12,53.0,1,2018
4,2018-01-01 13:53:00,29,1,Battle Axe,Combat Tools,95,10.0,13,120.00,44,874,Credit Card,1,48,250.0,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-06-29 17:48:00,80,11,Large Axe,Outdoor Equipment,75,6.0,2,155.11,0,310,Cash,180,32,246.0,6,2023
9996,2023-06-29 11:20:00,46,1,Battle Axe,Combat Tools,95,10.0,1,212.44,0,212,Cash,180,48,164.0,6,2023
9997,2023-06-29 15:24:00,44,1,Small Axe,Hand Tools,35,1.5,1,54.52,0,55,Credit Card,180,12,43.0,6,2023
9998,2023-07-02 10:28:00,38,6,Small Axe,Hand Tools,35,1.5,2,54.52,0,109,Credit Card,183,12,85.0,7,2023


In [69]:
print( data.groupby( 'product_name' ).size() )

product_name
Battle Axe    3354
Large Axe     3323
Medium Axe    1688
Small Axe     1635
dtype: int64


In [70]:
#Ktory dzien roku
data[ 'transaction_datetime' ] = pd.to_datetime( data[ 'transaction_datetime' ] )
data[ 'day_of_year' ] = data[ 'transaction_datetime' ].dt.dayofyear

In [71]:
print( data )
data.to_csv( 'output.csv', index = False )

     transaction_datetime  customer_id  location product_name  \
0     2018-01-01 13:46:00           80         3    Small Axe   
1     2018-01-01 09:53:00           91        12   Battle Axe   
2     2018-01-01 15:20:00           91        10   Medium Axe   
3     2018-01-01 09:58:00           23         7    Small Axe   
4     2018-01-01 13:53:00           29         1   Battle Axe   
...                   ...          ...       ...          ...   
9995  2023-06-29 17:48:00           80        11    Large Axe   
9996  2023-06-29 11:20:00           46         1   Battle Axe   
9997  2023-06-29 15:24:00           44         1    Small Axe   
9998  2023-07-02 10:28:00           38         6    Small Axe   
9999  2023-07-02 10:53:00           85        13    Small Axe   

               category  size_cm  weight_kg  quantity  unit_price  discount  \
0            Hand Tools       35        1.5        15       30.00        44   
1          Combat Tools       95       10.0        15      12

In [72]:
label_encoder = LabelEncoder()
data[ 'location' ] = label_encoder.fit_transform( data[ 'location' ] )
data[ 'product_name' ] = label_encoder.fit_transform( data[ 'product_name' ] )

In [73]:
data

,transaction_datetime,customer_id,location,product_name,category,size_cm,weight_kg,quantity,unit_price,discount,total_amount,payment_method,day_of_year,cost_price,profit,transaction_month,transaction_year
0,2018-01-01 13:46:00,80,3,3,Hand Tools,35,1.5,15,30.00,44,252,Credit Card,1,12,72.0,1,2018
1,2018-01-01 09:53:00,91,12,0,Combat Tools,95,10.0,15,120.00,44,1008,Credit Card,1,48,288.0,1,2018
2,2018-01-01 15:20:00,91,10,2,Hand Tools,55,3.0,13,50.00,44,364,Credit Card,1,20,104.0,1,2018
3,2018-01-01 09:58:00,23,7,3,Hand Tools,35,1.5,11,30.00,44,185,Cash,1,12,53.0,1,2018
4,2018-01-01 13:53:00,29,1,0,Combat Tools,95,10.0,13,120.00,44,874,Credit Card,1,48,250.0,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-06-29 17:48:00,80,11,1,Outdoor Equipment,75,6.0,2,155.11,0,310,Cash,180,32,246.0,6,2023
9996,2023-06-29 11:20:00,46,1,0,Combat Tools,95,10.0,1,212.44,0,212,Cash,180,48,164.0,6,2023
9997,2023-06-29 15:24:00,44,1,3,Hand Tools,35,1.5,1,54.52,0,55,Credit Card,180,12,43.0,6,2023
9998,2023-07-02 10:28:00,38,6,3,Hand Tools,35,1.5,2,54.52,0,109,Credit Card,183,12,85.0,7,2023


In [74]:
X = data[ [ 'unit_price', 'discount' ] ]
y = data[ 'quantity' ]

In [75]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 42 )

In [76]:
xg_reg = xgb.XGBRegressor( objective = 'reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 100 )

In [77]:
xg_reg.fit( X_train, y_train )

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [78]:
def predict_profits_in_date_range(data, product_name, new_price, start_date, end_date):
    data['transaction_month'] = data['transaction_datetime'].dt.month
    data['transaction_year'] = data['transaction_datetime'].dt.year
    filtered_data = data[(data['transaction_datetime'] >= start_date) & (data['transaction_datetime'] <= end_date)]

    historical_profits = filtered_data['profit']
    total_historical_profits = historical_profits.sum()

    filtered_data_copy = filtered_data.copy()
    filtered_data_copy.loc[filtered_data_copy['product_name'] == product_name, 'unit_price'] = new_price

    relevant_data = filtered_data_copy[filtered_data_copy['product_name'] == product_name]
    new_quantity_predictions = xg_reg.predict(relevant_data[['unit_price', 'discount']])

    new_quantity_predictions = new_quantity_predictions.astype(int)

    filtered_data_copy.loc[filtered_data_copy['product_name'] == product_name, 'quantity'] = new_quantity_predictions

    new_profits = (filtered_data_copy['unit_price'] * filtered_data_copy['quantity']) - (filtered_data_copy['cost_price'] * filtered_data_copy['quantity'])

    filtered_data_copy.loc[filtered_data_copy['product_name'] == product_name, 'profit'] = new_profits

    total_new_profits = filtered_data_copy[ 'profit' ].sum()

    return total_historical_profits, total_new_profits


In [79]:
new_price = 500000
product_name = 3
start_date = '2023-01-02'
end_date = '2023-01-03'

historical_profits, predicted_profits = predict_profits_in_date_range( data, product_name, new_price, start_date, end_date )

print(f"Historyczne zyski za okres od {start_date} do {end_date}: {historical_profits:.2f}")
print(f"Przewidywane zyski po podwyższeniu ceny: {predicted_profits:.2f}")

Historyczne zyski za okres od 2023-01-02 do 2023-01-03: 1156.00
Przewidywane zyski po podwyższeniu ceny: 1016.00


In [96]:
def predict_profits_future( data, product_name, new_price, cost, start_date, end_date ):
    future_dates = pd.date_range( start = start_date, end = end_date, freq = 'D' )
    
    future_data = pd.DataFrame( {
        'transaction_datetime': future_dates,
        'product_name': product_name,
        'unit_price': new_price,
        'discount': 0,
        'quantity': np.nan,
        'cost_price': cost,
        'profit': np.nan
    } )
    
    new_quantity_predictions = xg_reg.predict( future_data[ [ 'unit_price', 'discount' ] ] )
    new_quantity_predictions = new_quantity_predictions.astype(int)
    
    future_data['quantity'] = new_quantity_predictions
    
    future_data['profit'] = ( future_data['unit_price'] * future_data['quantity'] ) - ( future_data['cost_price'] * future_data['quantity'] )
    
    total_new_profits = future_data['profit'].sum()
    
    return total_new_profits

In [97]:
def find_optimal_price( data, product_name, cost, start_date, end_date, price_range ):
    lb = [np.min(price_range)]
    ub = [np.max(price_range)]

    optimal_price, max_profits = pso( objective_function, lb, ub, args = ( data, product_name, cost, start_date, end_date ) )

    return optimal_price[0], -max_profits

In [98]:
def objective_function( price, data, product_name, cost, start_date, end_date ):
    predicted_profits = predict_profits_future( data, product_name, price[0], cost, start_date, end_date )
    return -predicted_profits

In [100]:
today = date.today()
_, last_day_of_month = monthrange( today.year, today.month )
start_date = today
end_date = today.replace( day = last_day_of_month )

product_name = 3
cost = 60
price_range = range(10, 300, 1)

optimal_price, max_profits = find_optimal_price(data, product_name, cost, start_date, end_date, price_range)

print(f"Optymalna cena to: {optimal_price}")
print(f"Maksymalne przewidywane zyski: {max_profits:.2f}")

Stopping search: Swarm best position change less than 1e-08
Optymalna cena to: 186.429985007752
Maksymalne przewidywane zyski: 6068.64
